In [1]:
import os
import torch
import numpy as np
import mediapipe as mp

# Directories
processed_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/processed_npy"
keypoints_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/keypoints_gpu"
os.makedirs(keypoints_folder, exist_ok=True)

# Initialize CUDA device (for tensor storage only, MediaPipe remains on CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔹 Using device for tensor storage: {device}")

# Initialize MediaPipe Holistic (runs ONLY on CPU!)
mp_holistic = mp.solutions.holistic.Holistic(
    static_image_mode=False, model_complexity=1, smooth_landmarks=True, enable_segmentation=False
)

# Function to extract keypoints (Pose + Hands)
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33 * 3)
    left_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21 * 3)
    right_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21 * 3)
    
    return np.concatenate([pose, left_hand, right_hand])  # Total: 99 keypoints per frame

# Function to process a video (GPU for storage, CPU for MediaPipe)
def process_video(video_path, save_path):
    try:
        frames = np.load(video_path).astype(np.uint8)  # Convert to uint8

        keypoints_seq = []
        for frame in frames:
            frame_np = frame if isinstance(frame, np.ndarray) else frame.cpu().numpy()
            if frame_np.shape[0] == 3 and frame_np.shape[-1] != 3:
                frame_np = frame_np.transpose(1, 2, 0)  # [3, 224, 224] → [224, 224, 3]

            results = mp_holistic.process(frame_np)
            keypoints = extract_keypoints(results)
            keypoints_seq.append(keypoints)

        np.save(save_path, np.array(keypoints_seq))
        return True

    except Exception as e:
        print(f"❌ Error processing {video_path}: {e}")
        return False

# Process all videos
for video_file in os.listdir(processed_folder):
    if not video_file.endswith(".npy"):
        continue

    video_path = os.path.join(processed_folder, video_file)
    keypoint_path = os.path.join(keypoints_folder, video_file)

    if os.path.exists(keypoint_path):
        print(f"⏭️  Skipping {video_file} (already processed)")
        continue

    success = process_video(video_path, keypoint_path)

    if success:
        print(f"✅ Keypoints saved for {video_file}")

print("🚀 Keypoint extraction completed! (Processed only missing files)")

🔹 Using device for tensor storage: cuda


I0000 00:00:1741480122.862448 1018475 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741480122.905850 1018714 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.216.01), renderer: NVIDIA RTX A6000/PCIe/SSE2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1741480122.988406 1018648 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1741480123.037724 1018657 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1741480123.044034 1018653 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1741480123.047837 1018671 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Di

✅ Keypoints saved for 50037_secretary_44.npy
✅ Keypoints saved for 00314_sleep_89.npy
✅ Keypoints saved for 08672_california_33_brightness.npy
✅ Keypoints saved for 03058_appointment_56_slow.npy
✅ Keypoints saved for 00128_from_68_flip.npy
✅ Keypoints saved for 22086_first_61_fast.npy
✅ Keypoints saved for 00149_have_72_noise.npy
✅ Keypoints saved for 40119_orange_88_slow.npy
✅ Keypoints saved for 38524_no_80_brightness.npy
✅ Keypoints saved for 00154_hesheit_65_flip.npy
✅ Keypoints saved for 00405_what_52.npy
✅ Keypoints saved for 10112_chat_93_flip.npy
✅ Keypoints saved for 00334_some_64_noise.npy
✅ Keypoints saved for 04898_banana_56_fast.npy
✅ Keypoints saved for 66532_son_59_flip.npy
✅ Keypoints saved for 65341_cheat_76_slow.npy
✅ Keypoints saved for 69302_drink_77.npy
✅ Keypoints saved for 13635_cousin_64.npy
✅ Keypoints saved for 13198_cool_31_fast.npy
✅ Keypoints saved for 60354_ugly_44_brightness.npy
✅ Keypoints saved for 41452_past_31_rotate.npy
✅ Keypoints saved for 48517_ro